In [1]:
from known_boundary.GP import optimise,optimise_warp,optimise_warp_no_boundary
from known_boundary.utlis import Trans_function, get_initial_points,transform
from known_boundary.acquisition_function import EI_acquisition_opt,MES_acquisition_opt,Warped_EI_acquisition_opt,Warped_TEI1_acquisition_opt,Warped_TEI2_acquisition_opt,LCB_acquisition_opt,ERM_acquisition_opt
import numpy as np
import GPy
import torch
from botorch.test_functions import Ackley,Levy,Beale,Branin,Hartmann,Rosenbrock,Powell
from botorch.utils.transforms import unnormalize,normalize

import warnings
warnings.filterwarnings("ignore")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double

function_information = []


temp={}
temp['name']='Ackley12D' 
temp['function'] = Ackley(dim=12,negate=False)
temp['fstar'] = 0.
temp['min']=True 
function_information.append(temp)

/home/hanyang/anaconda3/envs/known_boundary/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
for information in function_information:

    fun = information['function']
    dim = fun.dim
    bounds = fun.bounds
    standard_bounds=np.array([0.,1.]*dim).reshape(-1,2)
    
    n_init = 4*dim
    iter_num = 300
    N = 1

    fstar = information['fstar']
    fun = Trans_function(fun,fstar,min=True)
    
    
    ################################################### GP+EI ###########################################
    BO_EI = []

    for exp in range(N):
        
        print(exp)
        
        seed = 98

        X_BO = get_initial_points(bounds, n_init,device,dtype,seed=seed)
        Y_BO = torch.tensor(
            [fun(x) for x in X_BO], dtype=dtype, device=device
        ).reshape(-1,1)

        best_record = [Y_BO.min().item()]
        np.random.seed(1234)

        for i in range(iter_num):
            
                print(i)
            
            
                train_Y = (Y_BO - Y_BO.mean()) / Y_BO.std()
                train_X = normalize(X_BO, bounds)
                
                minimal = train_Y.min().item()
                
                train_Y = train_Y.numpy()
                train_X = train_X.numpy()
                
                # train the GP
                if i%5 == 0:
                    print('tranin')
                    res = optimise(train_X,train_Y)
                    kernel = GPy.kern.RBF(input_dim=dim,lengthscale= np.sqrt(res[0]),variance=res[1]) 
                m = GPy.models.GPRegression(train_X, train_Y,kernel)
                m.Gaussian_noise.variance.fix(10**(-5))

                standard_next_X = EI_acquisition_opt(m,bounds=standard_bounds,f_best=minimal)
                X_next = unnormalize(torch.tensor(standard_next_X), bounds).reshape(-1,dim)            
                Y_next = fun(X_next).reshape(-1,1)

                # Append data
                X_BO = torch.cat((X_BO, X_next), dim=0)
                Y_BO = torch.cat((Y_BO, Y_next), dim=0)
                
                best_record.append(Y_BO.min().item())
                
                print(best_record[-1])
                
                
        best_record = np.array(best_record)+fstar 
        BO_EI.append(best_record)
        

0
0
tranin
18.581286155061928
1
17.783365021255424
2
17.226936042812905
3
17.226936042812905
4
17.226936042812905
5
tranin
17.16117730300142
6
17.16117730300142
7
17.07024356689553
8
16.084627515357113
9
15.3206154762802
10
tranin
15.3206154762802
11
15.3206154762802
12
15.3206154762802
13
15.3206154762802
14
14.721621626965852
15
tranin
14.026915178803433
16
14.026915178803433
17
14.026915178803433
18
14.026915178803433
19
14.026915178803433
20
tranin
14.026915178803433
21
14.026915178803433
22
13.693221223063478
23
13.693221223063478
24
13.139822142228883
25
tranin
12.358113141362818
26
11.61834455050705
27
11.61834455050705
28
11.61834455050705
29
11.58674117188852
30
tranin
10.979855207269626
31
10.342641971727332
32
10.342641971727332
33
9.27421134636157
34
8.650377782087414
35
tranin
8.650377782087414
36
8.650377782087414
37
8.650377782087414
38
7.074339107190376
39
7.074339107190376
40
tranin
7.074339107190376
41
7.074339107190376
42
7.074339107190376
43
7.074339107190376
44
7.0